# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Jun 27 2022 10:01AM,240500,10,CLI00312.1,"CLINUVEL, Inc.",Released
1,Jun 27 2022 10:01AM,240500,10,CLI00313.1,"CLINUVEL, Inc.",Released
2,Jun 27 2022 10:01AM,240500,10,CLI00314.1,"CLINUVEL, Inc.",Released
3,Jun 27 2022 9:59AM,240502,21,618729,"NBTY Global, Inc.",Released
4,Jun 27 2022 9:58AM,240501,21,618782,"NBTY Global, Inc.",Released
5,Jun 27 2022 9:54AM,240494,19,ELI06282022,Eli Lilly and Company,Released
6,Jun 27 2022 9:52AM,240499,21,618972,"NBTY Global, Inc.",Released
7,Jun 27 2022 9:52AM,240498,21,618468,"NBTY Global, Inc.",Released
8,Jun 27 2022 9:51AM,240496,20,SSNA-2173691916,Sartorius Stedim North America,Released
9,Jun 27 2022 9:51AM,240497,10,0085797460,ISDIN Corporation,Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
29,240498,Released,1
30,240499,Released,1
31,240500,Released,3
32,240501,Released,1
33,240502,Released,1


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Executing,Released
id,,
240498,NaN,1.0
240499,NaN,1.0
240500,NaN,3.0
240501,NaN,1.0
240502,NaN,1.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Executing,Released
id,,
240455,0.0,4.0
240463,26.0,21.0
240464,0.0,1.0
240466,0.0,58.0
240467,0.0,49.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

Column Completed does not exist yet


ShipmentStatus,Executing,Released
id,,
240455,0,4
240463,26,21
240464,0,1
240466,0,58
240467,0,49


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Executing,Released
0,240455,0,4
1,240463,26,21
2,240464,0,1
3,240466,0,58
4,240467,0,49


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

Column Completed does not exist yet


ShipmentStatus,id,Executing,Released
0,240455,,4
1,240463,26,21
2,240464,,1
3,240466,,58
4,240467,,49


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Executing    object
Released      int32
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Jun 27 2022 10:01AM,240500,10,"CLINUVEL, Inc."
3,Jun 27 2022 9:59AM,240502,21,"NBTY Global, Inc."
4,Jun 27 2022 9:58AM,240501,21,"NBTY Global, Inc."
5,Jun 27 2022 9:54AM,240494,19,Eli Lilly and Company
6,Jun 27 2022 9:52AM,240499,21,"NBTY Global, Inc."
7,Jun 27 2022 9:52AM,240498,21,"NBTY Global, Inc."
8,Jun 27 2022 9:51AM,240496,20,Sartorius Stedim North America
9,Jun 27 2022 9:51AM,240497,10,ISDIN Corporation
10,Jun 27 2022 9:49AM,240492,21,"NBTY Global, Inc."
11,Jun 27 2022 9:46AM,240491,20,"Exact-Rx, Inc."


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Executing,Released
0,Jun 27 2022 10:01AM,240500,10,"CLINUVEL, Inc.",,3
1,Jun 27 2022 9:59AM,240502,21,"NBTY Global, Inc.",,1
2,Jun 27 2022 9:58AM,240501,21,"NBTY Global, Inc.",,1
3,Jun 27 2022 9:54AM,240494,19,Eli Lilly and Company,,1
4,Jun 27 2022 9:52AM,240499,21,"NBTY Global, Inc.",,1
5,Jun 27 2022 9:52AM,240498,21,"NBTY Global, Inc.",,1
6,Jun 27 2022 9:51AM,240496,20,Sartorius Stedim North America,,1
7,Jun 27 2022 9:51AM,240497,10,ISDIN Corporation,,1
8,Jun 27 2022 9:49AM,240492,21,"NBTY Global, Inc.",,1
9,Jun 27 2022 9:46AM,240491,20,"Exact-Rx, Inc.",,1


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing
0,Jun 27 2022 10:01AM,240500,10,"CLINUVEL, Inc.",3,
1,Jun 27 2022 9:59AM,240502,21,"NBTY Global, Inc.",1,
2,Jun 27 2022 9:58AM,240501,21,"NBTY Global, Inc.",1,
3,Jun 27 2022 9:54AM,240494,19,Eli Lilly and Company,1,
4,Jun 27 2022 9:52AM,240499,21,"NBTY Global, Inc.",1,
5,Jun 27 2022 9:52AM,240498,21,"NBTY Global, Inc.",1,
6,Jun 27 2022 9:51AM,240496,20,Sartorius Stedim North America,1,
7,Jun 27 2022 9:51AM,240497,10,ISDIN Corporation,1,
8,Jun 27 2022 9:49AM,240492,21,"NBTY Global, Inc.",1,
9,Jun 27 2022 9:46AM,240491,20,"Exact-Rx, Inc.",1,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing
0,Jun 27 2022 10:01AM,240500,10,"CLINUVEL, Inc.",3,
1,Jun 27 2022 9:59AM,240502,21,"NBTY Global, Inc.",1,
2,Jun 27 2022 9:58AM,240501,21,"NBTY Global, Inc.",1,
3,Jun 27 2022 9:54AM,240494,19,Eli Lilly and Company,1,
4,Jun 27 2022 9:52AM,240499,21,"NBTY Global, Inc.",1,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

Column Completed does not exist yet


,Date,id,Warehouse,Customer,Released,Executing
0,Jun 27 2022 10:01AM,240500,10,"CLINUVEL, Inc.",3,NaN
1,Jun 27 2022 9:59AM,240502,21,"NBTY Global, Inc.",1,NaN
2,Jun 27 2022 9:58AM,240501,21,"NBTY Global, Inc.",1,NaN
3,Jun 27 2022 9:54AM,240494,19,Eli Lilly and Company,1,NaN
4,Jun 27 2022 9:52AM,240499,21,"NBTY Global, Inc.",1,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing
0,Jun 27 2022 10:01AM,240500,10,"CLINUVEL, Inc.",3,0.0
1,Jun 27 2022 9:59AM,240502,21,"NBTY Global, Inc.",1,0.0
2,Jun 27 2022 9:58AM,240501,21,"NBTY Global, Inc.",1,0.0
3,Jun 27 2022 9:54AM,240494,19,Eli Lilly and Company,1,0.0
4,Jun 27 2022 9:52AM,240499,21,"NBTY Global, Inc.",1,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing
Warehouse,,,
10,4328646,496,0.0
12,480927,22,26.0
15,240489,19,0.0
16,480948,2,0.0
19,480949,5,0.0
20,961942,21,0.0
21,1202492,5,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing
0,10,4328646,496,0.0
1,12,480927,22,26.0
2,15,240489,19,0.0
3,16,480948,2,0.0
4,19,480949,5,0.0
5,20,961942,21,0.0
6,21,1202492,5,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing
0,10,496,0.0
1,12,22,26.0
2,15,19,0.0
3,16,2,0.0
4,19,5,0.0
5,20,21,0.0
6,21,5,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,496.0
1,12,Released,22.0
2,15,Released,19.0
3,16,Released,2.0
4,19,Released,5.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,12,15,16,19,20,21
Status,,,,,,,
Executing,0.0,26.0,0.0,0.0,0.0,0.0,0.0
Released,496.0,22.0,19.0,2.0,5.0,21.0,5.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,12,15,16,19,20,21
0,Executing,0.0,26.0,0.0,0.0,0.0,0.0,0.0
1,Released,496.0,22.0,19.0,2.0,5.0,21.0,5.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,12,15,16,19,20,21
0,Executing,0.0,26.0,0.0,0.0,0.0,0.0,0.0
1,Released,496.0,22.0,19.0,2.0,5.0,21.0,5.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()